In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [2]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [3]:
inputLines = spark.sparkContext.textFile("train_full_parsed_clean3.1.csv")
#inputLines = spark.sparkContext.textFile("train_full_parsed_clean5.csv")

In [4]:
header=inputLines.filter(lambda l: "_id" in l)

In [5]:
header.collect()

['id,trip_duration,vendor_id,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,duration,speed,speed_great_circle,hour_count,passenger_count,pickup_hour,pickup_weekday,pickup_month,snow,holiday,distance,route_cooridnates']

In [6]:
linesnoheader = inputLines.subtract(header)

In [36]:
linesnoheader.take(2)

['id2860026,548,1,2016-06-06 14:03:24,-74.01502990722656,40.70608139038085,-74.0018539428711,40.73017120361328,2900.5280937740936,650.0,24.34204379562044,19.054564119683825,3782,1,14,0,6,0,0,3705.4,"[[-74.015028, 40.706081], [-74.015175, 40.705537], [-74.014794, 40.706809], [-74.015735, 40.707222], [-74.014729, 40.709827], [-74.012759, 40.717114], [-74.01093, 40.725807], [-74.010478, 40.729823], [-74.005376, 40.729377], [-74.002476, 40.730337], [-74.002199, 40.729693], [-74.001911, 40.73019]]"',
 'id3869910,2460,2,2016-05-22 00:03:13,-73.98358154296875,40.74414825439453,-73.97847747802734,40.66371154785156,8957.019009439498,1669.8,22.271414634146343,13.10783269674073,3408,1,0,6,5,0,0,15218.8,"[[-73.983553, 40.744136], [-73.984024, 40.743301], [-73.983127, 40.744535], [-73.973104, 40.740321], [-73.975047, 40.735716], [-73.973844, 40.731292], [-73.971872, 40.729099], [-73.971948, 40.726999], [-73.974467, 40.722333], [-73.975234, 40.717638], [-73.978021, 40.712124], [-73.980185, 40.71098]

In [7]:
#coord=pd.read_csv('route_coord_count.csv')
coord=pd.read_csv('newcounter.csv')

kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000,random_state=1234).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

In [8]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [9]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    line_cluster=[]
    speed_great_circle=line.split(',')[11]
    hour_count=line.split(',')[12]
    for i in coordinates:
        temp=re.split('\[|, |\]',i)
        long=float(temp[1])
        lat=float(temp[2])
        cluster_group=kmeans.predict([[long,lat]])
        line_cluster+=[cluster_group[0]]
        
    line_cluster=list(set(line_cluster))
    all_cluster=[ float(speed_great_circle) if i in line_cluster else np.nan for i in range(100)]
        
    
    
    

    return [int(hour_count)]+all_cluster

    

In [10]:
line='id2860026,548,1,2016-06-06 14:03:24,-74.01502990722656,40.70608139038085,-74.0018539428711,40.73017120361328,2900.5280937740936,650.0,24.34204379562044,19.054564119683825,3782,1,14,0,6,0,0,3705.4,"[[-74.015028, 40.706081], [-74.015175, 40.705537], [-74.014794, 40.706809], [-74.015735, 40.707222], [-74.014729, 40.709827], [-74.012759, 40.717114], [-74.01093, 40.725807], [-74.010478, 40.729823], [-74.005376, 40.729377], [-74.002476, 40.730337], [-74.002199, 40.729693], [-74.001911, 40.73019]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
line_cluster=[]
speed_great_circle=line.split(',')[11]
hour_count=line.split(',')[12]
for i in coordinates:
        temp=re.split('\[|, |\]',i)
        long=float(temp[1])
        lat=float(temp[2])
        cluster_group=kmeans.predict([[long,lat]])
        line_cluster+=[cluster_group[0]]
        
line_cluster=list(set(line_cluster))
all_cluster=[ float(speed_great_circle) if i in line_cluster else np.nan for i in range(100)]



In [11]:
all_cluster

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 19.054564119683825,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 19.054564119683825,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [12]:
coord = linesnoheader.map(topcoordinates)

In [13]:
feature=['hour_count']
colNames=feature+cluster_name

In [14]:
colNames

['hour_count',
 'C0',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 'C44',
 'C45',
 'C46',
 'C47',
 'C48',
 'C49',
 'C50',
 'C51',
 'C52',
 'C53',
 'C54',
 'C55',
 'C56',
 'C57',
 'C58',
 'C59',
 'C60',
 'C61',
 'C62',
 'C63',
 'C64',
 'C65',
 'C66',
 'C67',
 'C68',
 'C69',
 'C70',
 'C71',
 'C72',
 'C73',
 'C74',
 'C75',
 'C76',
 'C77',
 'C78',
 'C79',
 'C80',
 'C81',
 'C82',
 'C83',
 'C84',
 'C85',
 'C86',
 'C87',
 'C88',
 'C89',
 'C90',
 'C91',
 'C92',
 'C93',
 'C94',
 'C95',
 'C96',
 'C97',
 'C98',
 'C99']

In [15]:
train = coord.toDF(colNames)

In [16]:
#train.repartition(40).write.parquet('parquet_cluster_40_files')
train.repartition(1).write.csv('speed_cluster.csv')

In [21]:
train=pd.read_csv('speed_cluster.csv',names=colNames)

In [30]:
pd.set_option('max_columns', 200)
train.head(200)

,hour_count,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99
0,3782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.054564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.054564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3408,13.107833,NaN,NaN,13.107833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.107833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.107833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.107833,NaN,NaN,NaN,NaN,NaN,NaN,13.107833,NaN,NaN,13.107833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.107833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.910346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2418,NaN,NaN,22.301539,22.301539,22.301539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.301539,NaN,NaN,NaN,NaN,22.301539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.301539,NaN,NaN,NaN,NaN,NaN,22.301539,22.301539,NaN,NaN,NaN,NaN,22.301539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.301539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.301539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.301539,NaN,NaN
4,1197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.327729,16.327729,NaN,NaN,NaN,NaN,16.327729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.327729,NaN,NaN,NaN,16.327729,NaN,NaN,NaN,NaN,NaN
5,3074,23.663305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.663305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.663305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.663305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2973,NaN,NaN,NaN,11.501486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.501486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.501486,11.501486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.501486,NaN,NaN,NaN,NaN,11.501486,NaN,NaN,NaN,NaN,11.501486,NaN,NaN,NaN,NaN
7,598,NaN,NaN,NaN,11.833080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.833080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.833080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.833080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.833080,11.833080,11.833080,NaN,NaN,11.833080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.833080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2247,NaN,NaN,NaN,14.740044,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [24]:
df=train.groupby('hour_count').median()

In [35]:
df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99
hour_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,16.993869,16.702293,44.737695,15.649919,34.167358,11.218397,NaN,NaN,29.098628,12.799113,11.323320,19.690587,15.472231,6.857936,18.274273,19.574131,17.857401,28.269419,17.205760,17.910803,21.778143,17.042337,22.329088,9.007745,16.157773,43.980077,12.107311,13.329326,15.718325,23.655384,18.044349,14.446427,12.136290,0.000000,15.789443,9.851550,11.298523,NaN,15.296239,12.387411,16.862809,15.158360,18.821430,NaN,22.329088,13.082605,6.500220,9.107456,13.171862,21.802483,14.446427,16.018795,17.795673,14.034795,13.285404,12.458853,13.930707,27.509337,22.312696,44.737695,9.278645,12.533121,13.290419,39.452526,11.264316,12.327309,16.182420,12.477376,15.660586,44.431153,18.720248,14.922546,18.799498,10.308677,16.162263,18.756338,14.348771,16.515816,43.222459,10.982932,18.668220,16.737180,44.124612,18.625817,22.464327,17.139937,16.731223,15.184060,15.328145,16.066143,12.659259,36.490723,28.550588,17.434242,12.867483,11.595308,NaN,44.737695,12.847790,20.175549
1,20.462104,13.865835,0.000000,15.084914,26.474725,13.100921,NaN,39.571530,22.250019,14.064729,11.772172,19.147253,13.484977,18.370650,19.291695,17.354679,17.054994,25.600008,15.965144,16.939859,18.455048,13.972958,NaN,11.528820,14.308541,28.315783,15.515334,14.630459,16.739539,24.544515,15.511273,NaN,15.012524,39.571530,16.990803,10.279164,10.222782,26.060029,27.229929,11.850196,14.989706,13.100921,19.395469,NaN,14.184415,14.271805,31.607491,7.935446,14.461373,22.605882,17.226168,15.429192,15.306487,14.830927,14.210578,14.334096,15.179079,21.262328,23.649702,NaN,9.318877,11.612837,14.339915,33.870807,11.230257,14.256558,15.702615,15.684510,13.868223,39.847975,21.883302,15.429192,17.156494,14.458132,15.424277,16.479358,15.682107,16.246460,25.246697,11.509580,15.472800,14.392941,26.484373,18.581011,21.177932,17.175459,14.022255,14.271805,15.123480,15.965144,12.569439,NaN,26.474725,15.069088,14.619472,11.962828,NaN,NaN,17.320449,19.657126
2,23.469176,15.484696,28.538327,17.924120,31.447370,12.807128,NaN,NaN,29.082386,12.697642,13.042539,21.487927,12.645139,18.975455,19.206835,21.487927,19.827033,29.333507,18.183563,18.761083,23.400459,15.032188,29.331794,12.213073,15.312493,30.074982,12.822240,14.980290,18.736930,22.633770,14.460014,28.182398,14.940296,NaN,18.833052,11.047224,13.625524,34.250571,23.568476,14.963553,16.530321,16.568890,19.553661,41.472881,30.043029,17.672165,14.838211,13.050716,19.864695,25.494367,30.501078,18.457844,15.565256,16.244086,16.362709,14.470496,13.323481,29.082386,19.446784,28.538327,12.810178,12.742878,12.834739,30.074982,12.696638,15.501994,16.395279,16.060901,15.565256,26.475331,22.589917,18.544085,18.580363,12.917616,18.850263,19.097384,16.873545,18.862501,36.169393,11.349405,18.967545,15.956762,26.494428,28.225912,18.689330,18.828631,15.084295,18.403519,15.485636,19.380539,13.993142,31.938891,36.169393,15.635677,13.325174,12.625887,32.819758,31.917239,17.880973,21.487927
3,22.034213,18.583237,34.742861,17.132935,24.297324,15.501088,NaN,NaN,17.251612,12.945495,15.751211,23.897915,14.298092,9.216392,22.051181,21.972636,20.731490,25.898605,17.730668,19.434227,24.444551,15.313107,24.252503,11.774289,18.460958,25.822409,15.012648,17.390419,19.974499,23.049706,16.048195,23.163897,19.512561,NaN,17.154265,13.013781,12.989060,30.906242,18.856194,13.373730,19.895853,17.283376,22.909770,NaN,19.499766,20.143917,11.695634,12.595540,20.183720,25.022233,16.093749,15.313107,17.449028,20.594884,18.944493,17.291670,14.273183,25.171578,22.173831,34.742861,15.027382,12.588861,1

NameError: name 'median' is not defined